In [ ]:
import os
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
import tensorflow as tf
from keras.optimizers import Adam
import cv2
import numpy as np
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
import matplotlib.pyplot as plt
%matplotlib inline

# Running the model using CPU
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

In [ ]:
graph = tf.get_default_graph()

fc_layers = [1024, 1024]
dropout = 0.5
HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights= None, 
                      include_top=False,
                      input_shape=(HEIGHT, WIDTH, 3))



x = base_model.output
x = Flatten()(x)
for fc in fc_layers:
    x = Dense(fc, activation='relu')(x) 
    x = Dropout(dropout)(x)

predictions = Dense(10, activation='softmax')(x) 
    
finetune_model = Model(inputs=base_model.input, outputs=predictions)

    

#Load weights
finetune_model.load_weights('ResNet50_model_weights.h5')  

In [ ]:
class_list = ["ana","close","continue","eat","fen","listen","open","short",
              "tall","watch"]

finetune_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.1),
              metrics=['accuracy'])


def predict(img):
    img = cv2.resize(img,(300,300))
    img = np.reshape(img,[1,300,300,3])
    classes = finetune_model.predict(img)[0]
    #return class_list[classes.argmax()],classes
    return str(classes.argmax())

In [ ]:
app = Flask(__name__)
CORS(app)

@app.route('/', methods=['POST'])
def test():
    
    
    nparr = np.frombuffer(request.data, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    
    
    with graph.as_default():
        prediction = predict(img)
    ret = prediction
    return ret

# running REST interface, port=5000 for direct test
if __name__ == "__main__":
    app.run(debug=False, host='0.0.0.0', port=5000)